In [1]:
import os
import sys
import json
import datetime
import argparse
import logging
import math
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader, Subset, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms

# Import U-Mamba modules (assume nnunetv2 and dynamic_network_architectures are in PYTHONPATH)
from nnunetv2.nets.UMambaEnc_2d import get_umamba_enc_2d_from_plans
from nnunetv2.utilities.plans_handling.plans_handler import ConfigurationManager, PlansManager
from dynamic_network_architectures.building_blocks.helper import convert_dim_to_conv_op

/home/fonta42/Desktop/masters-degree/U-Mamba/umamba/nnunetv2/nets/UMambaEnc_2d.py:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [ ]:
# Dataset Definition
class VessMapDataset(Dataset):
    def __init__(self, image_dir, mask_dir, skeleton_dir, image_size, mode='train', apply_transform=False):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.skeleton_dir = skeleton_dir
        self.image_size = image_size
        self.mode = mode
        self.apply_transform_flag = apply_transform
        self.class_weights_tensor = torch.tensor([0.26, 0.74])

        # List all image files with .tiff or .tif extension
        self.image_files = [f for f in os.listdir(self.image_dir) if f.endswith('.tiff') or f.endswith('.tif')]
        self.image_files.sort()

        # Find matching pairs of images, masks, and skeletons
        self.pairs = []
        for img_file in self.image_files:
            base_name = os.path.splitext(img_file)[0]
            mask_file = base_name + ".png"
            skeleton_file = base_name + ".png"

            mask_path = os.path.join(self.mask_dir, mask_file)
            skeleton_path = os.path.join(self.skeleton_dir, skeleton_file)

            if os.path.exists(mask_path) and os.path.exists(skeleton_path):
                self.pairs.append((img_file, mask_file, skeleton_file))
            else:
                print(f"Warning: Missing mask or skeleton for image {img_file}")

        self.images = []
        self.labels = []
        self.skeletons = []
        for img_file, mask_file, skeleton_file in self.pairs:
            image_path = os.path.join(self.image_dir, img_file)
            mask_path = os.path.join(self.mask_dir, mask_file)
            skeleton_path = os.path.join(self.skeleton_dir, skeleton_file)

            image = Image.open(image_path).convert('RGB')
            mask = Image.open(mask_path).convert('L')
            skeleton = Image.open(skeleton_path).convert('L')

            self.images.append(image)
            self.labels.append(mask)
            self.skeletons.append(skeleton)

    def __len__(self):
        return len(self.pairs)

    def apply_transform(self, image, mask, skeleton, seed=42):
        random.seed(seed)
        torch.manual_seed(seed)
        if random.random() > 0.5:
            image = TF.hflip(image)
            mask = TF.hflip(mask)
            skeleton = TF.hflip(skeleton)
        if random.random() > 0.5:
            image = TF.vflip(image)
            mask = TF.vflip(mask)
            skeleton = TF.vflip(skeleton)
        angle = random.uniform(-30, 30)
        image = TF.rotate(image, angle)
        mask = TF.rotate(mask, angle)
        skeleton = TF.rotate(skeleton, angle)
        i, j, h, w = transforms.RandomCrop.get_params(image, output_size=(self.image_size, self.image_size))
        image = TF.crop(image, i, j, h, w)
        mask = TF.crop(mask, i, j, h, w)
        skeleton = TF.crop(skeleton, i, j, h, w)
        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask)
        skeleton = TF.to_tensor(skeleton)
        return image, mask, skeleton

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.labels[idx]
        skeleton = self.skeletons[idx]
        if self.mode == 'train' and self.apply_transform_flag:
            image, mask, skeleton = self.apply_transform(image, mask, skeleton)
        else:
            image = TF.to_tensor(image)
            mask = TF.to_tensor(mask)
            skeleton = TF.to_tensor(skeleton)
        return image, mask, skeleton

    def vess_map_dataloader(self, batch_size, train_size, shuffle=True):
        dataset_size = len(self)
        train_len = int(train_size * dataset_size)
        indices = list(range(dataset_size))
        train_indices = indices[:train_len]
        test_indices = indices[train_len:]
        train_dataset = Subset(VessMapDataset(self.image_dir, self.mask_dir, self.skeleton_dir, self.image_size, mode='train', apply_transform=self.apply_transform_flag), train_indices)
        test_dataset = Subset(VessMapDataset(self.image_dir, self.mask_dir, self.skeleton_dir, self.image_size, mode='test', apply_transform=False), test_indices)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        return train_loader, test_loader

In [ ]:
# Logging Utilities
LOG_RECORDS = []
def log_message(level, message):
    logger = logging.getLogger("UMamba_Training")
    logger.log(level, message)
    LOG_RECORDS.append({
        "level": logging.getLevelName(level),
        "message": message,
        "time": datetime.datetime.now().isoformat()
    })

# -------------------------------
# Dice Metric
# -------------------------------
def compute_dice(preds, targets):
    preds = preds.long()
    targets = targets.long()
    intersection = (preds & targets).sum(dim=(2, 3))
    dice = (2.0 * intersection.float()) / (preds.sum(dim=(2, 3)) + targets.sum(dim=(2, 3)) + 1e-6)
    return dice.mean().item()

# -------------------------------
# Difference Mask Visualization
# -------------------------------
def create_difference_mask(bin_preds, bin_targets):
    b, _, h, w = bin_preds.shape
    diff_masks = np.zeros((b, h, w, 3), dtype=np.uint8)
    for idx in range(b):
        bin_pred = bin_preds[idx, 0]
        bin_tgt  = bin_targets[idx, 0]
        tp = (bin_pred == 1) & (bin_tgt == 1)
        fp = (bin_pred == 1) & (bin_tgt == 0)
        fn = (bin_pred == 0) & (bin_tgt == 1)
        diff_masks[idx][tp] = [0, 255, 0]       # Green
        diff_masks[idx][fp] = [255, 255, 0]       # Yellow
        diff_masks[idx][fn] = [255, 0, 0]         # Red
    return diff_masks

def save_comparison_plot(original_img, bin_pred, diff_mask, epoch, batch_idx, model_name):
    if torch.is_tensor(original_img):
        original_img = original_img.cpu().numpy()
    if torch.is_tensor(bin_pred):
        bin_pred = bin_pred.cpu().numpy()
    original_img = np.transpose(original_img, (1, 2, 0))
    output_dir = os.path.join("umamba", "validation-diff-masks", f"{model_name}_validation-diff-masks")
    os.makedirs(output_dir, exist_ok=True)
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(original_img, cmap="gray") 
    axs[0].set_title("Original Image")
    axs[0].axis("off")
    axs[1].imshow(bin_pred, cmap="gray")
    axs[1].set_title("Prediction")
    axs[1].axis("off")
    axs[2].imshow(diff_mask)
    axs[2].set_title("Diff Mask")
    axs[2].axis("off")
    plt.tight_layout()
    save_path = os.path.join(output_dir, f"comparison_epoch{epoch}_batch{batch_idx}.png")
    plt.savefig(save_path)
    plt.close(fig)

# -------------------------------
# Loss Functions
# -------------------------------
def dice_loss(pred_logits, target):
    # Upsample predictions if needed to match target spatial dimensions.
    if pred_logits.shape[-2:] != target.shape[-2:]:
        pred_logits = F.interpolate(pred_logits, size=target.shape[-2:], mode='bilinear', align_corners=False)
    pred_prob = torch.sigmoid(pred_logits)
    pred_bin = (pred_prob > 0.5).float()
    target = target.float()
    intersection = torch.sum(pred_bin * target)
    union = torch.sum(pred_bin) + torch.sum(target)
    dice_coeff = (2.0 * intersection + 1e-8) / (union + 1e-8)
    return 1.0 - dice_coeff

def ce_loss_wrapper(pred, target):
    # Upsample pred if its spatial dimensions do not match the target's
    if pred.shape[-2:] != target.shape[-2:]:
        pred = F.interpolate(pred, size=target.shape[-2:], mode='bilinear', align_corners=False)
    return F.binary_cross_entropy_with_logits(pred, target)


def compute_loss(preds, masks, seg_loss_func, ce_loss_func, loss_type):
    # If preds is a list (deep supervision), compute loss on each output and average.
    if isinstance(preds, list):
        loss_total = 0.0
        for pred in preds:
            if loss_type == "dice":
                loss_total += seg_loss_func(pred, masks)
            elif loss_type == "ce":
                loss_total += ce_loss_wrapper(pred, masks.float())
            elif loss_type == "both":
                loss_total += seg_loss_func(pred, masks) + ce_loss_wrapper(pred, masks.float())
            else:
                raise ValueError(f"Unknown loss type: {loss_type}")
        return loss_total / len(preds)
    else:
        if loss_type == "dice":
            return seg_loss_func(preds, masks)
        elif loss_type == "ce":
            return ce_loss_wrapper(preds, masks.float())
        elif loss_type == "both":
            return seg_loss_func(preds, masks) + ce_loss_wrapper(preds, masks.float())
        else:
            raise ValueError(f"Unknown loss type: {loss_type}")

In [ ]:
# Training and Validation Loops
def train_one_epoch(model, dataloader, device, optimizer, seg_loss_func, ce_loss_func, loss_type, accumulate_grad_steps=1, clip_grad=True):
    model.train()
    running_loss = 0.0
    step_count = 0
    for step, (images, masks, _) in enumerate(tqdm(dataloader, desc="Training")):
        images = images.to(device)
        masks = masks.to(device)
        preds = model(images)
        loss = compute_loss(preds, masks, seg_loss_func, ce_loss_func, loss_type)
        loss.backward()
        if (step + 1) % accumulate_grad_steps == 0:
            if clip_grad:
                clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
        running_loss += loss.item()
        step_count += 1
    return running_loss / step_count

def validate(model, dataloader, device, seg_loss_func, ce_loss_func, loss_type, epoch=0, model_name="default_model"):
    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    count = 0
    with torch.no_grad():
        for batch_idx, (images, masks, _) in enumerate(tqdm(dataloader, desc="Validating")):
            images = images.to(device)
            masks = masks.to(device)
            preds = model(images)
            loss = compute_loss(preds, masks, seg_loss_func, ce_loss_func, loss_type)
            val_loss += loss.item()

            # If deep supervision returns a list, choose one output (e.g. the last)
            if isinstance(preds, list):
                pred_eval = preds[-1]
            else:
                pred_eval = preds

            # Upsample pred_eval if needed
            if pred_eval.shape[-2:] != masks.shape[-2:]:
                pred_eval = torch.nn.functional.interpolate(pred_eval, size=masks.shape[-2:], mode='bilinear', align_corners=False)

            probs = torch.sigmoid(pred_eval)
            bin_preds = (probs > 0.5).float()
            bin_masks = (masks > 0.5).float()
            dice_value = compute_dice(bin_preds, bin_masks)
            val_dice += dice_value
            count += 1

        if epoch % 50 == 0 and epoch != 0:
            # For visualization, again use the last output
            if isinstance(preds, list):
                pred_vis = preds[-1]
            else:
                pred_vis = preds
            if pred_vis.shape[-2:] != masks.shape[-2:]:
                pred_vis = torch.nn.functional.interpolate(pred_vis, size=masks.shape[-2:], mode='bilinear', align_corners=False)
            probs_vis = torch.sigmoid(pred_vis)
            bin_preds_vis = (probs_vis > 0.5).float()
            diff_masks = create_difference_mask(bin_preds_vis.cpu().numpy(), (masks > 0.5).float().cpu().numpy())
            first_original = images[0]
            first_bin_pred = bin_preds_vis[0, 0]
            first_diff_mask = diff_masks[0]
            save_comparison_plot(
                original_img=first_original, 
                bin_pred=first_bin_pred, 
                diff_mask=first_diff_mask, 
                epoch=epoch, 
                batch_idx=batch_idx,
                model_name=model_name
            )
    return val_loss / count, val_dice / count

In [ ]:
# Main Training Routine for U-Mamba
def train_umamba(model, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    log_message(logging.INFO, "Preparing dataset...")
    vess_dataset = VessMapDataset(
        image_dir=args.image_dir,
        mask_dir=args.mask_dir,
        skeleton_dir=args.skeleton_dir,
        image_size=args.image_size,
        apply_transform=args.augment
    )
    train_loader, test_loader = vess_dataset.vess_map_dataloader(batch_size=args.batch_size, train_size=args.train_size / 100)
    ce_loss_func = nn.BCEWithLogitsLoss(reduction="mean")
    seg_loss_func = dice_loss  # using our dice_loss as segmentation loss
    if args.optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)
    elif args.optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=args.lr)
    else:
        raise ValueError("Unknown optimizer choice. Choose from ['sgd', 'adam'].")
    if args.scheduler == "cosine":
        num_steps = args.epochs * len(train_loader)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)
    else:
        scheduler = None
    log_message(logging.INFO, "Training parameters:")
    for arg_name, val in vars(args).items():
        log_message(logging.INFO, f" - {arg_name}: {val}")
    now = datetime.datetime.now()
    date_str = now.strftime("%d%m%Y")
    model_name = f"umamba_{args.optimizer}_lr{args.lr}_bs{args.batch_size}_{args.loss_type}_{args.train_size}pct_{date_str}"
    train_losses = []
    val_losses = []
    val_dices = []
    best_loss = float("inf")
    for epoch in range(args.epochs):
        log_message(logging.INFO, f"Starting epoch {epoch+1}/{args.epochs}")
        train_loss = train_one_epoch(model=model, dataloader=train_loader, device=device, optimizer=optimizer, seg_loss_func=seg_loss_func, ce_loss_func=ce_loss_func, loss_type=args.loss_type, accumulate_grad_steps=args.accumulate_grad_steps, clip_grad=True)
        train_losses.append(train_loss)
        val_loss, val_dice = validate(model=model, dataloader=test_loader, device=device, seg_loss_func=seg_loss_func, ce_loss_func=ce_loss_func, loss_type=args.loss_type, epoch=epoch, model_name=model_name)
        val_losses.append(val_loss)
        val_dices.append(val_dice)
        log_message(logging.INFO, f"Epoch {epoch+1:02d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Dice: {val_dice:.4f}")
        if scheduler is not None:
            scheduler.step()
        if val_loss < best_loss:
            best_loss = val_loss
            os.makedirs("./umamba/models", exist_ok=True)
            torch.save(model.state_dict(), f"./umamba/models/{model_name}_best.pth")
            log_message(logging.INFO, f"Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}")
        torch.save(model.state_dict(), f"./umamba/models/{model_name}_latest.pth")
    metrics = {
        "train_losses": train_losses,
        "val_losses": val_losses,
        "val_dices": val_dices,
    }
    os.makedirs("./umamba/metrics", exist_ok=True)
    with open(f"./umamba/metrics/{model_name}_metrics.json", "w") as f:
        json.dump(metrics, f, indent=4)
    log_message(logging.INFO, f"Metrics saved to ./umamba/metrics/{model_name}_metrics.json")
    fig, ax1 = plt.subplots(figsize=(10, 5))
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='tab:blue')
    l1 = ax1.plot(range(1, args.epochs+1), train_losses, label='Train Loss', color='tab:blue', linestyle='-')
    l2 = ax1.plot(range(1, args.epochs+1), val_losses, label='Val Loss', color='tab:blue', linestyle='--')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax2 = ax1.twinx()
    ax2.set_ylabel('Val Dice', color='tab:red')
    l3 = ax2.plot(range(1, args.epochs+1), val_dices, label='Val Dice', color='tab:red', linestyle='-')
    ax2.tick_params(axis='y', labelcolor='tab:red')
    lines = l1 + l2 + l3
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper center')
    plt.title('Training & Validation Loss and Dice Over Epochs')
    plt.grid(True)
    plt.savefig(f"./umamba/metrics/{model_name}_training_validation_metrics.png")
    plt.close(fig)
    log_message(logging.INFO, f"Plot saved to ./umamba/metrics/{model_name}_training_validation_metrics.png")
    os.makedirs("./umamba/logs", exist_ok=True)
    with open(f"./umamba/logs/{model_name}_logs.json", "w") as f:
        json.dump(LOG_RECORDS, f, indent=4)
    log_message(logging.INFO, f"Logs saved to ./umamba/logs/{model_name}_logs.json")

In [ ]:
import sys
sys.path.append('/home/fonta42/Desktop/masters-degree/U-Mamba')
from umamba.nnunetv2.nets.UMambaEnc_2d import UMambaEnc
from dynamic_network_architectures.building_blocks.helper import convert_dim_to_conv_op

# Define your model parameters manually:
input_size = (256, 256)            # Input patch size
input_channels = 3                 # RGB images
n_stages = 4                       # Number of stages in the encoder/decoder
features_per_stage = [32, 64, 128, 256]  # Feature channels per stage
kernel_sizes = [[3, 3]] * n_stages       # 3x3 convolutions at each stage
# Define strides (e.g., no downsampling at first stage, then downsample by factor of 2)
strides = [[1, 1], [2, 2], [2, 2], [2, 2]]
n_conv_per_stage = 2               # Number of convolutional blocks per encoder stage
n_conv_per_stage_decoder = 2       # Number of convolutional blocks per decoder stage
num_classes = 1                    # For binary segmentation
conv_op = nn.Conv2d               # Use standard 2D convolutions
norm_op = nn.InstanceNorm2d       # InstanceNorm is commonly used
norm_op_kwargs = {'eps': 1e-5, 'affine': True}
nonlin = nn.LeakyReLU             # Activation function
nonlin_kwargs = {'inplace': True}
deep_supervision = True           # If you want deep supervision from the decoder
stem_channels = features_per_stage[0]  # The stem uses the first stage's feature count

# Create the U-Mamba model instance directly:
model = UMambaEnc(
    input_size=input_size,
    input_channels=input_channels,
    n_stages=n_stages,
    features_per_stage=features_per_stage,
    conv_op=conv_op,
    kernel_sizes=kernel_sizes,
    strides=strides,
    n_conv_per_stage=n_conv_per_stage,
    num_classes=num_classes,
    n_conv_per_stage_decoder=n_conv_per_stage_decoder,
    conv_bias=True,
    norm_op=norm_op,
    norm_op_kwargs=norm_op_kwargs,
    dropout_op=None,
    dropout_op_kwargs=None,
    nonlin=nonlin,
    nonlin_kwargs=nonlin_kwargs,
    deep_supervision=deep_supervision,
    stem_channels=stem_channels
)

# Move the model to your device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

feature_map_sizes: [[256, 256], [128, 128], [64, 64], [32, 32]]
do_channel_token: [False, False, False, False]
MambaLayer: dim: 64
MambaLayer: dim: 256


In [7]:
def get_args():
    parser = argparse.ArgumentParser(description="Train U-Mamba on Vessel Segmentation Dataset.")
    parser.add_argument("--lr", type=float, default=1e-3, help="Learning rate")
    parser.add_argument("--batch_size", type=int, default=8, help="Batch size")
    parser.add_argument("--epochs", type=int, default=50, help="Number of epochs")
    parser.add_argument("--optimizer", type=str, default="adam", choices=["sgd", "adam"], help="Optimizer choice")
    parser.add_argument("--loss_type", type=str, default="both", choices=["dice", "ce", "both"], help="Loss type")
    parser.add_argument("--scheduler", type=str, default="none", choices=["cosine", "none"], help="Scheduler type")
    parser.add_argument("--train_size", type=float, default=80, help="Train split ratio as a percentage")
    parser.add_argument("--image_size", type=int, default=256, help="Image size for data augmentation")
    parser.add_argument("--accumulate_grad_steps", type=int, default=1, help="Accumulate grad steps")
    parser.add_argument("--image_dir", type=str, default="../../data/vess-map/images", help="Images directory")
    parser.add_argument("--mask_dir", type=str, default="../../data/vess-map/labels", help="Masks directory")
    parser.add_argument("--skeleton_dir", type=str, default="../../data/vess-map/skeletons", help="Skeletons directory")
    parser.add_argument("--augment", type=bool, default=True, help="Apply random data augmentation")
    # Pass an empty list to parse_args so that defaults are used
    return parser.parse_args([])

# Get arguments (using defaults)
args = get_args()

# Setup logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
                    handlers=[logging.StreamHandler(sys.stdout)])

# Run your training function
train_umamba(model, args)

# Log completion
log_message(logging.INFO, "Training completed.")

2025-02-27 10:14:44,730 - UMamba_Training - INFO - Preparing dataset...
2025-02-27 10:14:45,041 - UMamba_Training - INFO - Training parameters:
2025-02-27 10:14:45,041 - UMamba_Training - INFO -  - lr: 0.001
2025-02-27 10:14:45,042 - UMamba_Training - INFO -  - batch_size: 8
2025-02-27 10:14:45,042 - UMamba_Training - INFO -  - epochs: 50
2025-02-27 10:14:45,042 - UMamba_Training - INFO -  - optimizer: adam
2025-02-27 10:14:45,043 - UMamba_Training - INFO -  - loss_type: both
2025-02-27 10:14:45,043 - UMamba_Training - INFO -  - scheduler: none
2025-02-27 10:14:45,043 - UMamba_Training - INFO -  - train_size: 80
2025-02-27 10:14:45,044 - UMamba_Training - INFO -  - image_size: 256
2025-02-27 10:14:45,044 - UMamba_Training - INFO -  - accumulate_grad_steps: 1
2025-02-27 10:14:45,044 - UMamba_Training - INFO -  - image_dir: ../../data/vess-map/images
2025-02-27 10:14:45,044 - UMamba_Training - INFO -  - mask_dir: ../../data/vess-map/labels
2025-02-27 10:14:45,045 - UMamba_Training - INFO

Validating: 100%|██████████| 3/3 [00:00<00:00, 13.62it/s]

2025-02-27 10:14:48,843 - UMamba_Training - INFO - Epoch 01, Train Loss: 0.8750, Val Loss: 0.6447, Val Dice: 0.7387
2025-02-27 10:14:48,873 - UMamba_Training - INFO - Best model saved at epoch 1 with val_loss 0.6447
2025-02-27 10:14:48,901 - UMamba_Training - INFO - Starting epoch 2/50



Validating: 100%|██████████| 3/3 [00:00<00:00, 14.00it/s]

2025-02-27 10:14:52,123 - UMamba_Training - INFO - Epoch 02, Train Loss: 0.4342, Val Loss: 0.5737, Val Dice: 0.7675
2025-02-27 10:14:52,155 - UMamba_Training - INFO - Best model saved at epoch 2 with val_loss 0.5737
2025-02-27 10:14:52,189 - UMamba_Training - INFO - Starting epoch 3/50


...


Validating: 100%|██████████| 3/3 [00:00<00:00, 12.21it/s]

2025-02-27 10:17:43,099 - UMamba_Training - INFO - Epoch 49, Train Loss: 0.1331, Val Loss: 0.3844, Val Dice: 0.8668
2025-02-27 10:17:43,132 - UMamba_Training - INFO - Starting epoch 50/50



Validating: 100%|██████████| 3/3 [00:00<00:00, 12.34it/s]

2025-02-27 10:17:46,934 - UMamba_Training - INFO - Epoch 50, Train Loss: 0.1259, Val Loss: 0.4075, Val Dice: 0.8623
2025-02-27 10:17:46,968 - UMamba_Training - INFO - Metrics saved to ./umamba/metrics/umamba_adam_lr0.001_bs8_both_80pct_27022025_metrics.json


2025-02-27 10:17:47,112 - UMamba_Training - INFO - Plot saved to ./umamba/metrics/umamba_adam_lr0.001_bs8_both_80pct_27022025_training_validation_metrics.png
2025-02-27 10:17:47,113 - UMamba_Training - INFO - Logs saved to ./umamba/logs/umamba_adam_lr0.001_bs8_both_80pct_27022025_logs.json
2025-02-27 10:17:47,115 - UMamba_Training - INFO - Training completed.
